In [1]:
%%capture

!pip install /kaggle/input/pytorch-geometric-packages/torch_scatter-2.1.2-cp310-cp310-linux_x86_64.whl
!pip install /kaggle/input/pytorch-geometric-packages/torch_sparse-0.6.18-cp310-cp310-linux_x86_64.whl
!pip install torch-geometric
!pip install ogb

In [2]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.data import DataLoader
from torch_geometric.datasets import QM9
from torch_geometric.transforms import NormalizeFeatures
import torch_geometric
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator
import torch_geometric.transforms as T

In [3]:
class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers,
                 dropout, return_embeds=False):
        super(GCN, self).__init__()
        self.num_layers = num_layers
        # A list of GCNConv layers
        convs_list = [GCNConv(hidden_dim, hidden_dim) for i in range(num_layers - 2)]
        convs_list.append(GCNConv(hidden_dim, output_dim))
        convs_list.insert(0, GCNConv(input_dim, hidden_dim))
        self.convs = torch.nn.ModuleList(convs_list)

        # A list of 1D batch normalization layers
        self.bns = torch.nn.ModuleList([torch_geometric.nn.BatchNorm(hidden_dim) for i in range(num_layers - 1)])

        # The log softmax layer
        self.softmax = torch.nn.LogSoftmax(dim=1)

        # Probability of an element getting zeroed
        self.dropout = dropout

        # Skip classification layer and return node embeddings
        self.return_embeds = return_embeds

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x, adj_t):
        out = None

        # hidden layers
        for i, conv in enumerate(self.convs[:-1]):
          conv1_result = conv(x, adj_t)
          batch1_result = self.bns[i](conv1_result)
          relu1_result = F.relu(batch1_result)
          x = F.dropout(relu1_result, p=self.dropout, training=self.training)

        # last layer
        x = self.convs[-1](x, adj_t)

        if self.return_embeds:
          out = x
        else:
          out = self.softmax(x)
        #########################################

        return out

In [5]:
dataset_name = 'ogbn-arxiv'
dataset = PygNodePropPredDataset(name=dataset_name,
                                 transform=T.ToSparseTensor())
data = dataset[0]

# Make the adjacency matrix to symmetric
# data.adj_t = data.adj_t.to_symmetric()

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# If you use GPU, the device should be cuda
print('Device: {}'.format(device))

data = data.to(device)
split_idx = dataset.get_idx_split()
train_idx = split_idx['train'].to(device)

/opt/conda/lib/python3.10/site-packages/torch_geometric/data/dataset.py:238: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  if osp.exists(f) and torch.load(f) != _repr(self.p

Device: cuda


In [6]:
def train(model, data, train_idx, optimizer, loss_fn):
    model.train()
    loss = 0

    optimizer.zero_grad()
    out = model(data.x, data.adj_t)
    loss = loss_fn(out[train_idx], data.y[train_idx].squeeze(1))
    #########################################

    loss.backward()
    optimizer.step()

    return loss.item()

In [7]:
# Test function here
@torch.no_grad()
def test(model, data, split_idx, evaluator, save_model_results=False):
    # TODO: Implement a function that tests the model by
    # using the given split_idx and evaluator.
    model.eval()

    # The output of model on all data
    out = None

    ############# Your code here ############
    ## (~1 line of code)
    ## Note:
    ## 1. No index slicing here
    out = model(data.x, data.adj_t)
    #########################################

    y_pred = out.argmax(dim=-1, keepdim=True)

    train_acc = evaluator.eval({
        'y_true': data.y[split_idx['train']],
        'y_pred': y_pred[split_idx['train']],
    })['acc']
    valid_acc = evaluator.eval({
        'y_true': data.y[split_idx['valid']],
        'y_pred': y_pred[split_idx['valid']],
    })['acc']
    test_acc = evaluator.eval({
        'y_true': data.y[split_idx['test']],
        'y_pred': y_pred[split_idx['test']],
    })['acc']

    if save_model_results:
      print ("Saving Model Predictions")

      data = {}
      data['y_pred'] = y_pred.view(-1).cpu().detach().numpy()

      df = pd.DataFrame(data=data)
      # Save locally as csv
      df.to_csv('ogbn-arxiv_node.csv', sep=',', index=False)


    return train_acc, valid_acc, test_acc

In [8]:
# Please do not change the args

args = {
    'device': device,
    'num_layers': 3,
    'hidden_dim': 256,
    'dropout': 0.5,
    'lr': 0.01,
    'epochs': 100,
}
args

{'device': 'cuda',
 'num_layers': 3,
 'hidden_dim': 256,
 'dropout': 0.5,
 'lr': 0.01,
 'epochs': 100}

In [9]:
model = GCN(data.num_features, args['hidden_dim'],
            dataset.num_classes, args['num_layers'],
            args['dropout']).to(device)

In [10]:
evaluator = Evaluator(name='ogbn-arxiv')

In [11]:
import copy
# reset the parameters to initial random value
model.reset_parameters()

optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])
loss_fn = F.nll_loss

best_model = None
best_valid_acc = 0

for epoch in range(1, 1 + args["epochs"]):
    loss = train(model, data, train_idx, optimizer, loss_fn)
    result = test(model, data, split_idx, evaluator)
    train_acc, valid_acc, test_acc = result
    if valid_acc > best_valid_acc:
        best_valid_acc = valid_acc
        best_model = copy.deepcopy(model)
    print(f'Epoch: {epoch:02d}, '
          f'Loss: {loss:.4f}, '
          f'Train: {100 * train_acc:.2f}%, '
          f'Valid: {100 * valid_acc:.2f}% '
          f'Test: {100 * test_acc:.2f}%')

Epoch: 01, Loss: 4.3172, Train: 1.87%, Valid: 2.26% Test: 2.48%
Epoch: 02, Loss: 2.8130, Train: 24.48%, Valid: 25.90% Test: 23.08%
Epoch: 03, Loss: 2.5188, Train: 32.83%, Valid: 31.67% Test: 29.65%
Epoch: 04, Loss: 2.3738, Train: 34.41%, Valid: 35.96% Test: 34.65%
Epoch: 05, Loss: 2.2591, Train: 32.26%, Valid: 33.07% Test: 33.82%
Epoch: 06, Loss: 2.1926, Train: 30.79%, Valid: 30.74% Test: 33.02%
Epoch: 07, Loss: 2.1217, Train: 30.92%, Valid: 30.27% Test: 33.13%
Epoch: 08, Loss: 2.0549, Train: 32.46%, Valid: 32.14% Test: 34.74%
Epoch: 09, Loss: 2.0074, Train: 35.47%, Valid: 35.95% Test: 37.55%
Epoch: 10, Loss: 1.9663, Train: 38.81%, Valid: 39.90% Test: 40.24%
Epoch: 11, Loss: 1.9295, Train: 41.89%, Valid: 43.14% Test: 42.11%
Epoch: 12, Loss: 1.8995, Train: 44.52%, Valid: 45.41% Test: 43.35%
Epoch: 13, Loss: 1.8702, Train: 46.93%, Valid: 47.41% Test: 44.15%
Epoch: 14, Loss: 1.8461, Train: 49.01%, Valid: 48.81% Test: 44.62%
Epoch: 15, Loss: 1.8230, Train: 50.71%, Valid: 50.05% Test: 45.07